In [1]:
from tsl.datasets import MetrLA, PemsBay, AirQuality
import pandas as pd
from os import path
import pickle
from deepecho import PARModel
dataset = PemsBay()
df3 = dataset.dataframe()[-1000:]
df3.columns = df3.columns.droplevel('channels')
df3.columns = [str(col) for col in df3.columns]

cols = list(df3.columns)
datetime = df3.index
df3 = pd.DataFrame(df3)
df3.columns = cols
df3.insert(0, 'Date', datetime)

/Users/paulmagos/.pyenv/versions/3.11.9/envs/ydata/lib/python3.11/site-packages/tsl/datasets/pems_bay.py:84: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/Users/paulmagos/.pyenv/versions/3.11.9/envs/ydata/lib/python3.11/site-packages/tsl/datasets/pems_bay.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0, inplace=True)


In [2]:
# Learn a model from the data
if path.exists('pam_pemsbay.pkl'):
    with open('pam_pemsbay.pkl', 'rb') as f:
        model = pickle.load(f)
else:
    model = PARModel(epochs=10, cuda=False)
    model.fit(
        data=df3,
        segment_size=1,
        sequence_index='Date'
    )
    with open('pam_pemsbay.pkl','wb') as f:
        pickle.dump(model,f)

Loss (0.004): 100%|██████████| 10/10 [17:02<00:00, 102.26s/it]


In [3]:
data_synth = model.sample(num_entities=1000)

100%|██████████| 1000/1000 [00:58<00:00, 17.01it/s]


In [4]:
data_synth

,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,409525,409526,409528,409529,413026,413845,413877,413878,414284,414694
0,67.770300,75.534545,65.889022,55.954200,54.749700,64.306100,54.131583,60.362100,63.703164,50.598200,...,65.128179,64.259300,57.860000,58.615090,65.797300,47.581494,63.984300,64.395000,70.159089,64.062900
1,64.582234,83.572252,58.606400,55.954200,50.709583,64.306100,53.359650,70.089180,58.039600,34.423918,...,65.488686,62.778735,57.860000,58.394044,65.797300,56.674600,62.030934,64.395000,66.723331,58.024772
2,67.770300,54.539500,58.606400,55.954200,53.115841,80.731138,56.436500,60.362100,64.218061,50.598200,...,63.459100,64.259300,57.860000,59.496800,65.797300,56.674600,63.984300,64.395000,65.476741,64.062900
3,67.770300,54.539500,61.445992,55.954200,54.749700,64.306100,56.436500,63.284346,58.039600,38.083912,...,59.309350,58.716867,51.459195,59.022613,65.797300,37.279172,55.365949,64.395000,65.423730,54.574371
4,67.770300,54.539500,58.606400,48.663008,50.095918,64.306100,56.436500,72.702645,52.164699,77.067593,...,63.901187,64.259300,57.438303,59.496800,65.797300,56.674600,55.629384,64.395000,68.200900,76.778191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,67.027204,54.539500,60.806374,60.545104,50.364931,64.306100,56.436500,60.362100,51.277665,73.268758,...,63.459100,56.796017,58.522905,59.496800,63.401385,56.674600,71.593578,62.905529,68.200900,64.062900
996,67.770300,54.019647,75.070326,56.183431,53.079299,58.957353,56.436500,64.078849,58.039600,40.421556,...,63.490274,72.027465,57.860000,59.639040,65.797300,47.315668,63.984300,64.395000,65.700376,64.062900
997,67.770300,59.864982,68.491682,55.954200,50.163987,63.910577,56.436500,60.362100,58.039600,59.934726,...,63.459100,64.259300,43.747283,55.238398,65.699844,67.113200,62.688823,65.307358,68.200900,59.974873
998,67.290151,54.564685,57.879588,55.954200,70.398431,64.306100,54.360491,60.362100,58.039600,50.598200,...,63.459100,64.259300,70.706263,59.496800,65.797300,61.535727,64.260390,64.395000,68.200900,68.285253


In [5]:
data_synth.to_csv('./synteticPemsBayPAR.csv', index=False)